In [1]:
import pandas as pd
import os

In [14]:
# Cargar el archivo de Excel
archivo_entrada = 'F:\Indice Diciembre OK.xlsx'

In [9]:
# Ruta de las carpetas que deseas explorar
ruta_carpeta = 'F:DICIEMBRE\Q2323'
ruta_carpeta_2 = 'F:\DICIEMBRE\Q2324'

In [15]:
# Leer el archivo de Excel y crear un DataFrame
df = pd.read_excel(archivo_entrada)

# Obtener la lista de nombres de archivos en la carpeta
nombres_archivos = os.listdir(ruta_carpeta)
# Obtener la lista de nombres de archivos en la segunda carpeta
nombres_archivos_2 = os.listdir(ruta_carpeta_2)

# Ruta de la segunda carpeta que deseas explorar
# ruta_carpeta_2 = '/ruta/a/la/segunda/carpeta'

# Seleccionar solo las columnas que deseas para trabajar
columnas_seleccionadas = ['QUINCENA', 'ANOMINA', 'REPORTE', 'NARCHIVO', 'ARCHIVO', 'DEPENDEN', 'SUB_DEP', 'VALIDACIÓN']
df = df[columnas_seleccionadas]

# Eliminar las columnas que no necesitas
columnas_a_eliminar = ['ANOMINA', 'NARCHIVO', 'ARCHIVO', 'VALIDACIÓN']
df = df.drop(columns=columnas_a_eliminar)

# Crear un DataFrame con los nombres de los archivos
df_archivos = pd.DataFrame({'Nombre de Archivo': nombres_archivos})

# Crear un DataFrame con los nombres de los archivos de la segunda carpeta
df_archivos_2 = pd.DataFrame({'Nombre de Archivo': nombres_archivos_2})

# Verificar si el valor en la columna "SUB_DEP" es un número y reemplazarlo por un valor nulo si no lo es
df['SUB_DEP'] = df['SUB_DEP'].apply(lambda x: str(x) if (isinstance(x, (int, float)) and not pd.isna(x)) else " ")

# Definir una función para generar la nomenclatura
def generar_nomenclatura(row):
    reporte = str(row['REPORTE'])  # Convertir el valor a cadena por si acaso
    dependen = str(row['DEPENDEN'])  # Convertir el valor a cadena por si acaso
    sub_dep = str(row['SUB_DEP']).split('.')[0]  # Convertir el valor a cadena y eliminar contenido desde el punto decimal
    primeros_tres = reporte[:3]
    primeros_tres = reporte[:3]
    ultimos_cuatro = reporte[-4:]
    cuarto_digito = reporte[3]
    guion_bajo = f"_{sub_dep}" if sub_dep.isdigit() else ''
    return f"{primeros_tres}_{ultimos_cuatro}_{cuarto_digito}_{dependen}{guion_bajo}"

# Aplicar la función para generar la columna de nomenclatura
df['Nomenclatura'] = df.apply(generar_nomenclatura, axis=1)

# Mostrar la información requerida junto con la nueva columna de nomenclatura
#print(df[['REPORTE', 'DEPENDEN', 'SUB_DEP', 'Nomenclatura']])
print(df.head()[['Nomenclatura']])

# Crear una nueva columna en el DataFrame con los nombres de las nomenclaturas
# df['Nomenclatura'] = df.apply(lambda row: f"{row['REPORTE'][:3]}_{row['REPORTE'][-4:]}_{row['REPORTE'][3]}_{row['DEPENDEN']}_{row['SUB_DEP']}" if isinstance(row['SUB_DEP'], int) else f"{row['REPORTE'][:3]}_{row['REPORTE'][-4:]}_{row['REPORTE'][3]}_{row['DEPENDEN']}" if pd.notnull(row['SUB_DEP']) else f"{row['REPORTE'][:3]}_{row['REPORTE'][-4:]}_{row['REPORTE'][3]}_{row['DEPENDEN']}_", axis=1)

# df['Nomenclatura'] = df.apply(lambda row: f"{row['REPORTE'][:3]}_{row['REPORTE'][-4:]}_{row['REPORTE'][3]}_{row['DEPENDEN']}_{'' if pd.isna(row['SUB_DEP']) or not str(row['SUB_DEP']).isdigit() else '_' + str(row['SUB_DEP'])}", axis=1)

# Quitar la extensión de los nombres de archivo en el DataFrame
df_archivos['Nombre de Archivo'] = df_archivos['Nombre de Archivo'].apply(lambda x: os.path.splitext(x)[0])

# Quitar la extensión de los nombres de archivo de la segunda carpeta
df_archivos_2['Nombre de Archivo'] = df_archivos_2['Nombre de Archivo'].apply(lambda x: os.path.splitext(x)[0])

# Concatenar los DataFrames de ambas carpetas
df_final = pd.concat([df_archivos, df_archivos_2], ignore_index=True)

# Renombrar la columna
df_final = df_final.rename(columns={'Nombre de Archivo': 'Nomenclatura'})

# Mostrar los primeros registros del nuevo DataFrame
print(df_final.head())
#print(df_final.tail(50))

# Mostrar los primeros registros del nuevo DataFrame
# print(df_archivos.head())

# Obtener el número de archivos en cada DataFrame
num_archivos_df_1 = df_archivos.shape[0]
num_archivos_df_2 = df_archivos_2.shape[0]
num_archivos_df_final = df_final.shape[0]
df_total = df.shape[0]

# Mostrar el número de archivos en cada DataFrame
print("Número de archivos en el primer DataFrame:", num_archivos_df_1)
print("Número de archivos en el segundo DataFrame:", num_archivos_df_2)
print("Número de archivos en el DataFrame final:", num_archivos_df_final)
print("Número de archivos DF total:", df_total)

     Nomenclatura
0  NOA_2323_0_657
1  NOD_2323_0_303
2  NOD_2323_0_307
3  NOA_2323_0_605
4  NOA_2323_0_304
                        Nomenclatura
0  ACTA ADMINISTRATIVA ANTICIPO_232_
1            ACTA ADMINISTRATIVA_105
2            ACTA ADMINISTRATIVA_201
3            ACTA ADMINISTRATIVA_224
4            ACTA ADMINISTRATIVA_232
Número de archivos en el primer DataFrame: 1043
Número de archivos en el segundo DataFrame: 907
Número de archivos en el DataFrame final: 1950
Número de archivos DF total: 1655


In [16]:
# Generar conjuntos de nomenclaturas para cada DataFrame
nomenclaturas_df = set(df['Nomenclatura'])
nomenclaturas_df_final = set(df_final['Nomenclatura'])

# Encontrar nomenclaturas que faltan en df_final
faltan_en_df_final = nomenclaturas_df - nomenclaturas_df_final

# Encontrar nomenclaturas que faltan en df
faltan_en_df = nomenclaturas_df_final - nomenclaturas_df

# Mostrar el número de archivos que faltan en cada DataFrame
print("Número de archivos que faltan en el DataFrame final:", len(faltan_en_df_final))
print("Número de archivos que faltan en el DataFrame original:", len(faltan_en_df))

# Mostrar la lista de nomenclaturas que no hacen match
print("Archivos que estan en el indice pero no estan escaneados:")
for nomenclatura in faltan_en_df_final:
    print(nomenclatura)

print("\nNomenclaturas en el Indice:")
for nomenclatura in faltan_en_df:
    print(nomenclatura)

Número de archivos que faltan en el DataFrame final: 10
Número de archivos que faltan en el DataFrame original: 493
Archivos que estan en el indice pero no estan escaneados:
NOA_2324_3_252
NOD_2324_H_211
NOA_2324_E_603
NOA_2324_E_252
NOA_2323_4_260
NOA_2323_3_252
NOD_2324_G_211
NOA_2324_E_222
NOA_2324_F_603
NOA_2324_F_252

Nomenclaturas en el Indice:
NOA_2324_0_650_3
NOA_2323_9_503
NOA_2324_F_503
NOD_2324_0_103
NOA_2323_A_232_1
NOA_2323_GE_305
NOD_2323_0_502
NOA_2324_3_603_4
NOA_2324_7_106
NOA_2323_A_509
NOA_2324_8_106
NOA_2323_4_508
NOA_2323_0_603_5
NOA_2324_5_603_6
NOA_2324_D_512
NOD_2323_A_512_1
NOA_2324_0_511
NOA_2324_0_603_4
NOA_2324_D_656_1
NOD_2323_B_231_3
NOD_2324_0_105
NOA_2324_9_513
NOA_2323_GE_223
NOA_2323_9_502
NOA_2323_7_609_1
NJA_2323_A_242_1
NOA_2324_B_242_1
NOD_2323_0_231_2
NOA_2324_0_501_3
NOA_2323_GE_506
NOA_2324_A_106
NOA_2324_7_252_1
NOA_2324_0_107
NOA_2323_9_513
NOA_2324_0_656_1
NOA_2324_5_503
NOA_2323_7_509
NOD_2324_2_101
NOA_2323_3_509
NOA_2324_B_503_1
NOD_2323_0

In [17]:
# Convertir las columnas de nomenclatura en conjuntos
nomenclaturas_df = set(df['Nomenclatura'])
nomenclaturas_df_final = set(df_final['Nomenclatura'])

# Encontrar nomenclaturas que faltan en df_final
faltan_en_df_final = nomenclaturas_df - nomenclaturas_df_final

# Mostrar el número de archivos que faltan en el DataFrame final
print("Número de archivos que faltan en el DataFrame final:", len(faltan_en_df_final))

# Mostrar nomenclaturas que faltan en el DataFrame final
print("Nomenclaturas que faltan en el DataFrame final:")
for nomenclatura in faltan_en_df_final:
    print(nomenclatura)

Número de archivos que faltan en el DataFrame final: 10
Nomenclaturas que faltan en el DataFrame final:
NOA_2324_3_252
NOD_2324_H_211
NOA_2324_E_603
NOA_2324_E_252
NOA_2323_4_260
NOA_2323_3_252
NOD_2324_G_211
NOA_2324_E_222
NOA_2324_F_603
NOA_2324_F_252
